In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import pandas as pd
import numpy as np
import argparse
import random
from model import KGCN

In [4]:
parser = argparse.ArgumentParser()

parser.add_argument('--dataset', type=str, default='movie', help='which dataset to use')
parser.add_argument('--aggregator', type=str, default='sum', help='which aggregator to use')
parser.add_argument('--n_epochs', type=int, default=10, help='the number of epochs')
parser.add_argument('--neighbor_sample_size', type=int, default=4, help='the number of neighbors to be sampled')
parser.add_argument('--dim', type=int, default=32, help='dimension of user and entity embeddings')
parser.add_argument('--n_iter', type=int, default=2, help='number of iterations when computing entity representation')
parser.add_argument('--batch_size', type=int, default=65536, help='batch size')
parser.add_argument('--l2_weight', type=float, default=1e-7, help='weight of l2 regularization')
parser.add_argument('--lr', type=float, default=2e-2, help='learning rate')
parser.add_argument('--ratio', type=float, default=1, help='size of training dataset')

args = parser.parse_args(['--lr', '2e-2'])

In [66]:
class KGCNDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        user_id = np.array(self.df.iloc[idx]['userId'])
        movie_id = np.array(self.df.iloc[idx]['movieId'])
        label = np.array(self.df.iloc[idx]['label'])
        return user_id, movie_id, label

In [67]:
train_dataset = KGCNDataset(x)
test_dataset = KGCNDataset(x_test)

In [68]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=2)

In [2]:
import torch.optim as optim

net = KGCN(num_user, num_entity, num_relation, kg, args)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

NameError: name 'KGCN' is not defined

In [ ]:
for i, (user_id, movie_id, label) in enumerate(train_loader):
    print(user_id, movie_id, label)
    if i > 10:
        break
    loss = 0
    print(net(user_id, movie_id))
    
    